In [1]:
import kfp
import kfp.dsl as dsl
import kfp.onprem as onprem
import kfp.components as comp


    
def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "$0"', text],
    )  

@dsl.pipeline(
    name='FMnistPipeline',
    description='mnist '
)
def fmnist_pipeline(learning_rate, dropout_rate, epoch, act, layer,  
                    checkpoint_dir, saved_model_dir, pvc_name, tensorboard_log,
                    name, model_version, namespace):
  
    exit_task = echo_op("Done!")
    
    with dsl.ExitHandler(exit_task): 

        kubeflow_pvc = dsl.PipelineVolume(pvc=str(pvc_name))
        
        mnist = dsl.ContainerOp(
            name='FMnist',
            image='jaewoo201/katib-job:55B18E86',
            command=['python', '/app/fmnist-save-model-renew.py'],
            arguments=[
                "--learning_rate", learning_rate,
                "--dropout_rate", dropout_rate,
                "--epoch", epoch,
                "--act", act,
                "--layer", layer,
                "--checkpoint_dir", checkpoint_dir,
                "--saved_model_dir", saved_model_dir,
                "--model_version", model_version,
                "--tensorboard_log", tensorboard_log
            ],
            pvolumes={"/result": kubeflow_pvc}
        )
        
        result = dsl.ContainerOp(
            name='list_list',
            image='library/bash:4.4.23',
            command=['ls', '-R', '/result'],
            pvolumes={"/result": mnist.pvolume}
        )
        
        kfserving = dsl.ContainerOp(
            name='kfserving',
            image='jaewoo201/kfserving:CCADDF13',
            command=['python', '/app/KFServing-fairing.py'],
            arguments=[
                "--namespace", namespace,
                "--storage_uri", "pvc://" +  str(pvc_name) + "/saved_model",
                "--name", name
            ]
          )        
                
        mnist_web_ui = dsl.ContainerOp(
            name='mnist_web_ui',
            image='brightfly/fmnist-webui-deploy:latest',
        )                
        
        result.after(mnist)
        kfserving.after(result)
        mnist_web_ui.after(kfserving)
    

arguments = {'learning_rate': '0.001397',
             'dropout_rate': '0.18',
             'epoch' : '1',
             'act' : 'sigmoid',
             'layer': '2',
             'checkpoint_dir': '/reuslt/training_checkpoints',
             'saved_model_dir':'/result/saved_model/',
             'pvc_name' : 'kubeflow-pvc',
             'tensorboard_log': '/result/log',
             'name' : 'kfserving-fmnist',
             'model_version' : '0001',
             'namespace' : 'kubeflow'
            }
    
if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(pipeline_func=fmnist_pipeline, 
                                               arguments=arguments)